In [1]:
from xgboost import XGBClassifier
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
np.random.seed(42)

In [3]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1)

In [4]:
mnist

{'data':        pixel1  pixel2  pixel3  pixel4  ...  pixel781  pixel782  pixel783  pixel784
 0         0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0
 1         0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0
 2         0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0
 3         0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0
 4         0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0
 ...       ...     ...     ...     ...  ...       ...       ...       ...       ...
 69995     0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0
 69996     0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0
 69997     0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0
 69998     0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0
 69999     0.0     0.0     0.0     0.0  ...       0.0       0.0     

In [6]:
type(mnist['data'])

pandas.core.frame.DataFrame

In [7]:
x = np.array(mnist['data'])
y = np.array(mnist['target'])

In [19]:
y

array(['5', '0', '4', ..., '4', '5', '6'], dtype=object)

In [24]:
x_train, x_test = x[:60000], x[60000:]
y_train, y_test = y[:60000], y[60000:]

In [20]:
y

array(['5', '0', '4', ..., '4', '5', '6'], dtype=object)

In [25]:
np.random.seed(42)
shuffled_index = np.random.permutation(60000)
shuffled_index

array([12628, 37730, 39991, ...,   860, 15795, 56422])

In [26]:
x_train = x_train[shuffled_index]
y_train = y_train[shuffled_index]
x_valid, y_valid = x_train[:5000], y_train[:5000]

In [28]:
xgbc = XGBClassifier()
xgbc.fit(x_train,y_train)

/usr/lib/python3.9/site-packages/xgboost/sklearn.py:1176: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[10:11:13] WARNING: /var/tmp/pamac-build-athharv/xgboost-git/src/xgboost/src/learner.cc:1094: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

rf = RandomForestClassifier()
sgd = SGDClassifier()
rf.fit(x_train,y_train)
sgd.fit(x_train,y_train)

SGDClassifier()

In [31]:
from sklearn.metrics import accuracy_score
y_predict = xgbc.predict(x_test)
rf_y_predict = rf.predict(x_test)
sgd_predict =  sgd.predict(x_test)
print('Accuracy Sore'.center(50,'-'))
print('XGB Accuracy =', accuracy_score(y_test,y_predict))
print('Random Forest Accuracy =', accuracy_score(y_test,rf_y_predict))
print('SGD Classifier Accuracy =', accuracy_score(y_test,sgd_predict))

------------------Accuracy Sore-------------------
XGB Accuracy = 0.978
Random Forest Accuracy = 0.9711
SGD Classifier Accuracy = 0.8334


In [32]:
from sklearn.model_selection import GridSearchCV

In [33]:
GridSearchCV?

Init signature:
GridSearchCV(
    estimator,
    param_grid,
    *,
    scoring=None,
    n_jobs=None,
    refit=True,
    cv=None,
    verbose=0,
    pre_dispatch='2*n_jobs',
    error_score=nan,
    return_train_score=False,
)
Docstring:     
Exhaustive search over specified parameter values for an estimator.

Important members are fit, predict.

GridSearchCV implements a "fit" and a "score" method.
It also implements "score_samples", "predict", "predict_proba",
"decision_function", "transform" and "inverse_transform" if they are
implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized
by cross-validated grid-search over a parameter grid.

Read more in the :ref:`User Guide <grid_search>`.

Parameters
----------
estimator : estimator object.
    This is assumed to implement the scikit-learn estimator interface.
    Either estimator needs to provide a ``score`` function,
    or ``scoring`` must be passed.

param_grid : dict or list o

In [36]:
param_grid = [{'learning_rate':[0.1,0.3],'min_child_weight': [0.5,0.2], 'gamma':[0., 0.2]}]
xgbc_grid = XGBClassifier(gpu_id = 0)
grid_search = GridSearchCV(xgbc_grid, param_grid, cv = 5, scoring = 'neg_mean_squared_error')
grid_search.fit(x_train,y_train)

/usr/lib/python3.9/site-packages/xgboost/sklearn.py:1176: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[10:42:19] WARNING: /var/tmp/pamac-build-athharv/xgboost-git/src/xgboost/src/learner.cc:1094: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/usr/lib/python3.9/site-packages/xgboost/sklearn.py:1176: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when

KeyboardInterrupt: 